In [6]:
import logging
import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import pandas as pd
import panel as pn
import panel.widgets as pnw
import numpy as np
import warnings
import xarray as xr
from matplotlib.lines import Line2D


def get_data(var_or_idx: str, feature: str):

    p = "./data/" + feature + "/<var_or_idx>.csv"
    df = pd.read_csv(p.replace("<var_or_idx>", var_or_idx))

    return df


def get_title(var_or_idx: str):
    
    title = ""
    if var_or_idx == "tas":
        title = "Température moyenne"
    elif var_or_idx == "tasmin":
        title = "Température minimale quotidienne"
    elif var_or_idx == "tasmax":
        title = "Température maximale quotidienne"
    elif var_or_idx == "pr":
        title = "Précipitation"
    elif var_or_idx == "evspsbl":
        title = "Évapotranspiration"
    elif var_or_idx == "evspsblpot":
        title = "Évapotranspiration potentielle"
        
    return title


def get_label(var_or_idx: str):
    
    units = ""
    if var_or_idx in ["tasmin", "tasmax", "tas"]:
        units = "Température (°C)"
    elif var_or_idx in ["pr"]:
        units = "Précipitation (mm)"
    elif var_or_idx == "evspsbl":
        units = "Évapotranspiration (mm)"
    elif var_or_idx == "evspsblpot":
        units = "Évapotranspiration (mm)"
        
    return units


def plot_ts():

    ref   = "ref"
    rcp26 = "rcp26"
    rcp45 = "rcp45"
    rcp85 = "rcp85"
    col_ref   = "black"
    col_rcp26 = "blue"
    col_rcp45 = "green"
    col_rcp85 = "red"

    # Load data.
    var = vars.value
    df = get_data(var, "time_series")

    # Extract data from CSV.
    data_year = df.year
    data_ref = []
    if ref in df.columns:
        data_ref = df.ref
    data_rcp26 = []
    if rcp26 + "_moy" in df.columns:
        data_rcp26 = [df.rcp26_min, df.rcp26_moy, df.rcp26_max]
    data_rcp45 = []
    if rcp45 + "_moy" in df.columns:
        data_rcp45 = [df.rcp45_min, df.rcp45_moy, df.rcp45_max]
    data_rcp85 = []
    if rcp85 + "_moy" in df.columns:
        data_rcp85 = [df.rcp85_min, df.rcp85_moy, df.rcp85_max]

    # Fonts.
    fs_sup_title = 9
    title = get_title(var)
    ylabel = get_label(var)

    # Initialize plot.
    fig = plt.figure(constrained_layout=True, figsize=(9, 5))
    specs = gridspec.GridSpec(ncols=1, nrows=1, figure=fig)
    ax = fig.add_subplot(specs[:])
    ax.set_title(title, fontsize=fs_sup_title)
    ax.set_xlabel("Année")
    ax.secondary_yaxis("right")
    ax.get_yaxis().tick_right()
    ax.axes.get_yaxis().set_visible(False)
    secax = ax.secondary_yaxis("right")
    secax.set_ylabel(ylabel)
    plt.subplots_adjust(top=0.925, bottom=0.10, left=0.03, right=0.90, hspace=0.30, wspace=0.416)

    # Loop through RCPs.
    rcps = [ref, rcp26, rcp45, rcp85]
    for rcp in rcps:

        # Skip if no simulation is available for this RCP.
        if ((rcp == ref) and (len(data_ref) == 0)) or \
           ((rcp == rcp26) and (len(data_rcp26) == 0)) or \
           ((rcp == rcp45) and (len(data_rcp45) == 0)) or \
           ((rcp == rcp85) and (len(data_rcp85) == 0)):
            continue

        # Colors.
        color = "black"
        if rcp == ref:
            color = col_ref
        elif rcp == rcp26:
            color = col_rcp26
        elif rcp == rcp45:
            color = col_rcp45
        elif rcp == rcp85:
            color = col_rcp85

        # Add data.
        if rcp == ref:
            ax.plot(data_year, data_ref, color="black", alpha=1.0)
        else:
            if rcp == rcp26:
                data_mean = data_rcp26[0]
                data_min  = data_rcp26[1]
                data_max  = data_rcp26[2]
            elif rcp == rcp45:
                data_mean = data_rcp45[0]
                data_min  = data_rcp45[1]
                data_max  = data_rcp45[2]
            elif rcp == rcp85:
                data_mean = data_rcp85[0]
                data_min  = data_rcp85[1]
                data_max  = data_rcp85[2]
            ax.plot(data_year, data_mean, color=color, alpha=1.0)
            ax.fill_between(np.array(data_year), data_min, data_max, color=color, alpha=0.25)

    # Finalize plot.
    legend_l = ["Référence"]
    if rcp26 in rcps:
        legend_l.append("RCP 2.6")
    if rcp45 in rcps:
        legend_l.append("RCP 4.5")
    if rcp85 in rcps:
        legend_l.append("RCP 8.5")
    custom_lines = [Line2D([0], [0], color="black", lw=4)]
    if rcp26 in rcps:
        custom_lines.append(Line2D([0], [0], color="blue", lw=4))
    if rcp45 in rcps:
        custom_lines.append(Line2D([0], [0], color="green", lw=4))
    if rcp85 in rcps:
        custom_lines.append(Line2D([0], [0], color="red", lw=4))
    ax.legend(custom_lines, legend_l, loc="upper left", frameon=False)
    # plt.ylim(ylim[0], ylim[1])
    plt.close(fig)
    return fig
    
    
def plot_ts_update(event):
    ts[0] = plot_ts()

    
def button_save_ts():
    pass


def button_save_map():
    pass


def button_save_stats():
    pass


def button_clear():
    pass

In [11]:
def test_matplotlib():
    
    logging.getLogger().disabled = True
    warnings.simplefilter("ignore")
    pn.extension()

    button_save_ts = pnw.Button(name='png', width=70)
    button_save_map = pnw.Button(name='png', width=70)
    button_clear = pnw.Button(name='png', width=70)

    logo_oura = "./data/ouranos.png"
    logo_oura1 = pn.Column(pn.pane.PNG(logo_oura,height=50))

    # Variables.
    var_list = ["tasmin", "tasmax", "pr", "evspsbl", "evspsblpot"]
    vars = pnw.Select(options=var_list, width=450)
    vars1 = pn.Column(pn.pane.Markdown('**Variable**'), vars)
    vars1

    # Horizons.
    # TODO: Detect automatically.
    hors_list = ["2021-2050", "2051-2080"]
    hors = pnw.DiscreteSlider(options=hors_list, value=hors_list[0], width=175, value_throttled=True, tooltips=True)
    hors1 = pn.Column(pn.pane.Markdown('**Horizon**'), hors)

    # RCP.
    # TODO: Detect automatically.
    rcps_list = ["RCP 2.6", "RCP 4.5","RCP 8.5", "Tous"]
    rcps = pnw.RadioButtonGroup(options=rcps_list, value=rcps_list[0], width=225)
    rcps1 = pn.Column(pn.pane.Markdown("**Scénario d'émissions**"),rcps)

    delta = pnw.Checkbox(value=False)
    delta1 = pn.Row(pn.pane.Markdown('**Afficher Δ**'), delta)

    # Latitude and longitude ranges.
    # TODO: Detect automatically.
    lat_range = (12.0, 14.5)
    lon_range = (-17.5, -11.0)
    lats = pnw.FloatInput(name='Latitude', value=None, step=0.10, start=lat_range[0], end=lat_range[1], width=100)
    lats.value = None
    lons = pnw.FloatInput(name='Longitude', value=None, step=0.10, start=lon_range[0], end=lon_range[1], width=100)
    lons.value = None
    lons1 = pn.Column(pn.pane.Markdown('**Coordonnées**'),lons)

    xclim_title = "## Informations et méthodologie"    
    xclim_desc = f"Les calculs de variables et indices climatiques ont été effectués en utilisant la librairie python \
      <a href='https://xclim.readthedocs.io/en/stable/' target='_blank'>**xclim**</a> \
      et sont résumés dans les tableaux. Les données sources pour les calculs de projections futurs correspondent à l'ensemble \
      <a href='https://www.csag.uct.ac.za/cordex-africa/' target='_blank'>**CORDEX-Afrique**</a> \
      et sont détaillés plus en détail ci-dessous. Les courbes d'observations dans les graphiques sont produites à partir des \
      données de réanalyses \
      <a href='https://www.ecmwf.int/en/era5-land' target='_blank'>**ERA5-Land**</a>."

    infos = pn.Column(pn.pane.Markdown(xclim_title), pn.pane.Markdown(xclim_desc), width=1200)

    ts = pn.Column(pn.Row(plot_ts()), pn.Row(pn.Column(pn.pane.Markdown("**Sauvegarder**"), button_save_ts), pn.Spacer(width=50)))
    map = pn.Column(pn.Row(pn.Column(pn.pane.Markdown("**Sauvegarder**"), button_save_map), pn.Spacer(width=50)))
    stats = pn.Column(pn.Row(pn.Column(pn.pane.Markdown("**Sauvegarder**"), button_save_stats), pn.Spacer(width=50)))

    tabs = pn.Tabs(("Séries temporelles", ts), ("Carte", map), ("Statistiques", stats), ("Infos & Méthodes", infos), dynamic=True)

    watch_vars = vars.param.watch(plot_ts_update, ["value"], onlychanged=True)

    controls = pn.Row(pn.Column(pn.Row(vars1), pn.Row(hors1, rcps1), delta1), pn.Column(lons1, lats, button_clear))

    dash = pn.Column(pn.Row(logo_oura1, pn.pane.Markdown("##Sénégal FAR")),
                     controls, tabs)
    display(dash)

In [ ]:
def test_nbinteract():
    
    die = Table().with_column('Face', np.arange(1, 7, 1))
    die

In [10]:
# Export to html.
# fname = "./data/dashboard.html"
# dash.save(fname)

In [9]:
# Declare the object as servable.
# Must comment this line (in a code cell above):
#   $ display (app).

# app.servable()

# Run using:
#   $ conda install dask
#   $ conda install -c bokeh jupyter_bokeh
#   $ panel serve --show scen_workflow_afr.ipynb

In [5]:
# Create dynamic webpage.

# $ pip install nbinteract
# The next two commands can be skipped for notebook version 5.3 and above:
# $ jupyter nbextension enable --py --sys-prefix bqplot
# $ jupyter nbextension enable --py --sys-prefix widgetsnbextension
